In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import seaborn as sns

## Setup & Basic info

In [ ]:
PATH = os.path.join("..", "data", "raw", "Allegato 1 - data_classification.xlsx")

df = pd.read_excel(PATH)
df.head()

In [ ]:
print("Shape:", df.shape)
df.info()
df.describe(include='all')

## Label Analysis

In [ ]:
text_col = "Review"
label_col = "Promotore"

df[label_col].value_counts().plot(kind='bar', figsize=(6,4), title="Label Distribution")
plt.show()

print(df[label_col].value_counts(normalize=True))

## Review Text Analysis

In [ ]:
# Identify empty reviews (only whitespace or truly empty after stripping)
empty_reviews = df[df[text_col].astype(str).str.strip().eq("")]

# Identify very short reviews (fewer than 3 characters).
short_reviews = df[df[text_col].astype(str).str.len() < 3]

print("Empty reviews:", len(empty_reviews))
print("Short reviews:", len(short_reviews))

In [ ]:
df["char_len"] = df[text_col].astype(str).str.len()
df["word_len"] = df[text_col].astype(str).str.split().str.len()

fig, ax = plt.subplots(1, 2, figsize=(12,4))

# Plot histogram of character lengths
sns.histplot(df["char_len"], bins=50, ax=ax[0])
ax[0].set_title("Character Length Distribution")
ax[0].set_xlabel("Characters")
ax[0].set_ylabel("Count")

# Plot histogram of word counts
sns.histplot(df["word_len"], bins=50, ax=ax[1])
ax[1].set_title("Word Count Distribution")
ax[1].set_xlabel("Words")
ax[1].set_ylabel("Count")

# Render the plots
plt.tight_layout()
plt.show()

# Show basic summary statistics for both length columns
df[["char_len", "word_len"]].describe()

In [ ]:
# Display the 5 longest reviews by character length
df.nlargest(5, "char_len")[[text_col, "char_len"]]

In [ ]:
df['review_length'] = df[text_col].str.len()

# Define the percentiles we want to inspect
percentiles = [0.75, 0.80, 0.90, 0.95, 0.99]

# Compute the quantile values for the specified percentiles
quantile_values = df['review_length'].quantile(percentiles)

# Print a compact table of percentile -> character length
print("--- Review Length Percentiles (Character Length) ---")
print(quantile_values)

# Print a more human-readable formatted output (rounded and shown as integers)
print("\nFormatted Output:")
for percentile, length in quantile_values.items():
	print(f"{int(percentile * 100):>2d}th Percentile: {int(round(length)):>4d} characters")

In [ ]:
# Analyze length distributions by label to check for possible correlations
df["char_len"] = df[text_col].astype(str).str.len()
df["word_len"]  = df[text_col].astype(str).str.split().str.len()

fig, ax = plt.subplots(1, 2, figsize=(12,5))

sns.boxplot(data=df, x=label_col, y="char_len", ax=ax[0])
ax[0].set_title("Character length by label")

sns.boxplot(data=df, x=label_col, y="word_len", ax=ax[1])
ax[1].set_title("Word count by label")

plt.show()

print(df.groupby(label_col)[["char_len", "word_len"]].describe())

In [ ]:
# Check for duplicate review texts
dups = df.duplicated(subset=[text_col], keep=False)
dups_df = df.loc[dups]
dups_counts = dups_df[text_col].value_counts()

print("--- Duplicate Review Text and Appearance Count ---")
print(dups_counts) 
print(f"\nTotal unique reviews that are duplicates: {len(dups_counts)}")
print(f"Total duplicate reviews: {dups.sum()}")

In [ ]:
# Check for conflicting labels among duplicates
dup_groups = dups_df.groupby(text_col)

# Initialize counters
conflicting_review_count = 0  # Counts unique reviews (text strings) with conflicts
total_conflict_instances = 0  # Counts total rows involved in conflicts

# Iterate through each group and check for label diversity
for review, group in dup_groups:
    # Check if there is more than one unique label in the group
    if group[label_col].nunique() > 1:
        # Increment the count of unique conflicting reviews
        conflicting_review_count += 1
        
        # Add the size of the entire group (all duplicate instances) 
        # to the total conflict instances count
        total_conflict_instances += len(group)

print("--- Conflicting Label Analysis ---")
print(f"Number of Unique Reviews with Conflicting Labels: {conflicting_review_count}")
print(f"Total Data Rows Involved in Label Conflicts: {total_conflict_instances}")

### Special Characters Presence

In [ ]:
url_pattern = re.compile(r"http\S+|www\.\S+")
hashtag_pattern = re.compile(r"#\w+")
mention_pattern = re.compile(r"@\w+")
html_pattern = re.compile(r"<[^>]+>")
emoji_pattern = re.compile(
    "[" 
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags
    "]+", 
    flags=re.UNICODE
)


In [ ]:
def contains(pattern, text):
    if not isinstance(text, str): 
        return False
    return bool(pattern.search(text))

df["has_url"] = df[text_col].apply(lambda x: contains(url_pattern, x))
df["has_hashtag"] = df[text_col].apply(lambda x: contains(hashtag_pattern, x))
df["has_mention"] = df[text_col].apply(lambda x: contains(mention_pattern, x))
df["has_html"] = df[text_col].apply(lambda x: contains(html_pattern, x))
df["has_emoji"] = df[text_col].apply(lambda x: contains(emoji_pattern, x))
df["punct_ratio"] = df[text_col].astype(str).apply(
    lambda x: sum(1 for c in x if c in r"!?,.;:") / max(len(x), 1)
)

In [ ]:
feature_cols = ["has_url", "has_hashtag", "has_mention", "has_html", "has_emoji"]

for col in feature_cols:
    print(f"{col}: {df[col].mean()*100:.2f}% of reviews")